### Illustration of symmetric key encryption using python
We show a simple example of the encrypt and decrypt operations using the classical Caesar shift cipher and the modern Advanced Encryption System (AES), which has been the standard for symmetric key encryption since 2001.

First we set up some Python libraries that provide the needed symmetric key encryption ciphers, and then define the plain text we wish to encrypt.

In [1]:
# Install the library if needed
%pip install secretpy

# import the required crypto functions which will be demonstrated later
from secretpy import Caesar
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from functools import reduce
import numpy as np

# Set the plaintext we want to encrypt
plaintext=u"this is a strict top secret message for intended recipients only"
print(f"\nGiven plaintext: {plaintext}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 6.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.

Given plaintext: this is a strict top secret message for intended recipients only


We will see how to encrypt and decrypt it using two different symmetric key encryption methods:

1. The classic Caesar shift cipher
2. The modern Advanced Encryption Standard AES-256 protocol

**Caesar shift cipher:**

Caesar shift encryption involves defining

An alphabet of possible characters to encode
A shift value which can be between 0 (unencrypted) and the length of the alphabet. We consider this the key.
It is known as a monoalphabetic substitution cipher since each letter of the plain text is substituted with another in the ciphertext.

In this example we will use lowercase letters of the alphabet.

Let's start by setting things up.

In [2]:
# initialize the required python object for doing Caesar shift encryption
caesar_cipher = Caesar()

# Define the shift, ie the key
caesar_key = 5 
print(f"Caesar shift secret key: {caesar_key}")

# Define the alphabet
alphabet=('a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ')
print(f"alphabet: {alphabet}")

Caesar shift secret key: 5
alphabet: ('a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ')


In [3]:
# Encrypt the plain text to get ciphertext for the Caesar cipher.
caeser_ciphertext = caesar_cipher.encrypt(plaintext, caesar_key, alphabet)
print(f"Encrypted caeser shift ciphertext: {caeser_ciphertext}")

Encrypted caeser shift ciphertext: ymnxenxefexywnhyeytuexjhwjyerjxxfljektwensyjsijiewjhnunjsyxetsqc


In [4]:
# Decrypt the ciphertext back to the original plain text using the same key used for encryption.
caeser_plaintext = caesar_cipher.decrypt(caeser_ciphertext, caesar_key, alphabet)
print(f"Decrypted caeser shift plaintext: {caeser_plaintext}\n")

Decrypted caeser shift plaintext: this is a strict top secret message for intended recipients only



**Advanced encryption standard (AES) cipher**

We now encrypt the plain text using AES, a popular symmetric key encryption algorithm.

We start by creating the key, in this case, a random 16-letter string.

In [5]:
# lamba defines an inline function in this case that takes two values a,b with the resulting expression of a+b
# reduce uses a two-argument function(above), and applies this to all the entries in the list (random alphabet characters) cumulatively
aes_key = reduce(lambda a, b: a + b, [np.random.choice(alphabet) for i in range(16)])

print(f'AES secret key: {aes_key}')

AES secret key: lwcrinew flfctfb


AES supports multiple operating modes and requires we specify which to use.

We choose the Cipher Block Chaining (CBC) mode provided by the modes.CBC class of the cryptography library. The CBC mode of AES uses randomness for additional security. This requires specifying a random Initialization Vector (IV), also called a nonce. We will use a random string for this as well, just like we did for the key.

In [6]:
aes_initialization_vector = reduce(lambda a, b: a + b, [np.random.choice(alphabet) for i in range(16)])
print(f"AES initialization vector: {aes_initialization_vector}")

AES initialization vector: aqunccp mnzrtwbc


We can now instantiate an AES cipher on behalf of the sender of the secret message. Note that the initialization vector is passed to the modes.CBC class to set up the CBC mode of operation.

We will then encrypt the plain text to send.

In [7]:
# The encryptor is setup using the key & CBC. In both cases we need to convert the string (utf-8) into bytes
sender_aes_cipher = Cipher(algorithms.AES(bytes(aes_key, 'utf-8')), modes.CBC(bytes(aes_initialization_vector, 'utf-8')))
aes_encryptor = sender_aes_cipher.encryptor()

# update can add text to encypt in chunks, and then finalize is needed to complete the encryption process
aes_ciphertext = aes_encryptor.update(bytes(plaintext, 'utf-8')) + aes_encryptor.finalize()

# Note the output is a string of bytes
print(f"Encrypted AES ciphertext: {aes_ciphertext}")

Encrypted AES ciphertext: b'\xf9\xe6\rh\xba]\x91\x82\xbb\x1c\xce\xce\xfa\x1ek}\xddv3vK\x03$\x14{\xdb\x81\x18 \x99\x1b#\x03\xc1\x8c\x04\x10\xfe\x15-#\xa1\xd6\x8e\xeaR\xad\xc3\x93<\xfc\x07\xcf\xd8\xf6\xcb*\x11\x0f\xd4\x95\xc0\rO'


To decrypt it, let us instantiate an AES cipher on behalf of the receiver. Note that the intended receiver has access to both the secret key and the initialization vector, but the latter is not required to be secret.

In [8]:
# Similar setup of AES to what we did for encryption, but this time, for decryption
receiver_aes_cipher = Cipher(algorithms.AES(bytes(aes_key, 'utf-8')), modes.CBC(bytes(aes_initialization_vector, 'utf-8')))
aes_decryptor = receiver_aes_cipher.decryptor()

# Do the decryption
aes_plaintext_bytes = aes_decryptor.update(aes_ciphertext) + aes_decryptor.finalize()

# convert back to a character string (we assume utf-8)
aes_plaintext = aes_plaintext_bytes.decode('utf-8')

print(f"Decrypted AES plaintext: {aes_plaintext}")

Decrypted AES plaintext: this is a strict top secret message for intended recipients only


### Applications of symmetric key cryptography
While classical ciphers such as the Caesar cipher fell out of use a long time ago, modern symmetric cryptosystems such as AES are deployed in a wide range of applications, including:

**1.Data encryption and decryption:** SKC is widely used to protect sensitive data, whether statically stored on a device or transmitted over a network. Examples include protecting user credentials, encrypting email messages, and securing financial transactions, among others.

**2.Secure communication:** Common communication protocols such as SSL/TLS use a combination of symmetric and asymmetric key encryption to ensure the confidentiality and integrity of data exchanged between two parties. These messages are encrypted and decrypted using symmetric key encryption which uses a shared key. The key used in symmetric key encryption is securely exchanged using asymmetric key encryption which uses a public-private key pair. Symmetric key encryption is much faster and hence can be used for encryption of messages of large size.

**3.Authenticity verification:** In some settings, SKC is employed through techniques like message authentication codes (MACs) and keyed-hash MACs (HMAC) to verify the authenticity and integrity of messages, ensuring tamper-resistant communication.

**4.File and disk encryption:** Full-disk encryption software and file encryption tools employ SKC to protect sensitive data stored on hard disks or portable storage devices.

**5.Virtual private networks:** VPN technologies, which aim to provide confidential communication channels free from eavesdropping, can use symmetric or asymmetric key encryption to connect remote users as well as corporate networks.

The diverse array of applications in which SKC is deployed in turn require that symmetric cryptosystems satisfy a certain set of criteria.